<a href="https://colab.research.google.com/github/hemanthkumar17/Image-Processing-Lab/blob/main/IP_Lab_cheatsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def fft1d(m):
  fft1d = np.zeros([m, m], dtype=complex)
  for u in range(m):
    for x in range(m):
      fft1d[u][x] = np.round(np.exp(-1j*2*np.pi*x*u/m), 10)
  ifft1d = np.conj(fft1d)
  return fft1d, ifft1d

In [ ]:
def dct1d(N):
  dct_d = np.zeros([N, N])

  for k in range(N):
    for n in range(N):
      dct_d[k, n] = np.cos(np.pi*((n+0.5)/N * k))
  return dct_d

In [ ]:
def walsh1d(N):
  def getBinaries(n):
    binary = [int(x) for x in bin(n)[2:]]
    return [0]*(int(np.log2(N))-len(binary)+1) + binary
  G1d = np.zeros([N, N])
  for x in range(N):
    for u in range(N):
      bx = getBinaries(x)
      bu = getBinaries(u)
      G1d[x, u] = (-1) ** sum(np.multiply(bx, bu[::-1]))
  return G1d

In [ ]:
def hadamard1d(N):
  def getBinaries(n):
    binary = [int(x) for x in bin(n)[2:]]
    return [0]*(int(np.log2(N))-len(binary)+1) + binary
  H1d = np.zeros([N, N])
  for x in range(N):
    for u in range(N):
      bx = getBinaries(x)
      bu = getBinaries(u)
      H1d[x, u] = (-1) ** sum(np.multiply(bx, bu))
  return H1d

In [ ]:
def haar1DBasis(N):
  val = np.zeros((N**2, 2))
  for k in range(1, N**2):
    val[k][0] = np.floor(np.log2(k))
    val[k][1] = k - 2**val[k][0] + 1

  haar1d = np.zeros((N, N))
  haar1d[0] = [1] * N
  for k in range(1, N):
    for m in range(N):
      t = m/N
      p, q = val[k]
      p2 = 2 ** p
      print(k, t, p, q)
      print((q - 0.5)/ p2, q/ p2, (q - 1)/ p2)
      if t >= (q - 0.5)/ p2 and t < q/ p2:
        haar1d[k, m] = -np.sqrt(p2)
      elif t >= (q - 1)/ p2 and t < (q - 0.5)/ p2:
        haar1d[k, m] = np.sqrt(p2)
  return haar1d

In [ ]:
def convolve_2D(image, kernel):
    return ccorrelation_2d(image, np.flipud(np.fliplr(kernel)))

def ccorrelation_2d(image, kernel):
    paddingl = kernel.shape[0] - 1
    paddingu = kernel.shape[1] - 1
    # Shape of Output Convolution
    xOutput = int(((image.shape[0] - kernel.shape[0] + 2 * paddingl) ) + 1)
    yOutput = int(((image.shape[1] - kernel.shape[1] + 2 * paddingu) ) + 1)
    output = np.zeros((xOutput, yOutput))

    # Apply Equal Padding to All Sides
    imagePadded = np.pad(image, ((paddingl, paddingl), (paddingu, paddingu)))
    print(imagePadded)

    # Iterate through image
    for y in range(yOutput):
        for x in range(xOutput):
            output[x, y] = np.sum(kernel * imagePadded[x: x + kernel.shape[0], y: y + kernel.shape[1]])
    return output

In [ ]:
convolve_2D(np.array([[1, 2, 1], [1, 2, 1], [3, 3, 3]]), np.array([[1, 1], [-1, -1]]))

[[0 0 0 0 0]
 [0 1 2 1 0]
 [0 1 2 1 0]
 [0 3 3 3 0]
 [0 0 0 0 0]]


array([[ 1.,  3.,  3.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 2.,  3.,  3.,  2.],
       [-3., -6., -6., -3.]])

In [ ]:
robertsFilterx = np.array([[1, 0], [0, -1]])
robertsFiltery = np.flipud(robertsFilterx).transpose()

sobelFilterx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
sobelFiltery = np.flipud(sobelFilterx).transpose()

prewittFilterx = np.array([[-1, 0, 1]]*3)
prewittFiltery = np.flipud(prewittFilterx).transpose()
print(prewittFiltery)

[[-1 -1 -1]
 [ 0  0  0]
 [ 1  1  1]]


In [ ]:
def histEqualization(img, max):
  hist, _ = np.histogram(img.flatten(), bins=range(np.max(img) + 2))
  pdf = hist / (img.shape[0] * img.shape[1])
  cdf = np.zeros(np.max(img) + 2)
  for i in range(np.max(img) + 1):
    cdf[i] = cdf[i-1] + pdf[i]
    cdf[i] = 1 if cdf[i] > 1 else cdf[i]
  cdf = cdf * max
  cdf = np.floor(cdf)
  new_image = np.zeros(img.shape)
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      new_image[i, j] = cdf[img[i, j]]
  return new_image, cdf

In [ ]:
img = np.array([[3, 2, 4, 5], [7, 7, 8, 2], [3, 1, 2, 3], [5, 4, 6, 7]])
print(histEqualization(img, 20))

(array([[ 8.,  5., 11., 13.],
       [18., 18., 20.,  5.],
       [ 8.,  1.,  5.,  8.],
       [13., 11., 15., 18.]]), array([ 0.,  1.,  5.,  8., 11., 13., 15., 18., 20.,  0.]))


In [ ]:
def histSpecialisation(origImage, targetImage):
  equal1 = histEqualization(origImage)[1]
  equal2 = histEqualization(targetImage)[1]
  return list(map(lambda eq1val: np.where(equal2 >= eq1val)[0][0], equal1))

In [ ]:
def imgClip(img, thresh):
  img[img<thresh] = 0
  return img
def imgWindow(img, window):
  img[img>window] = 0
  return img

In [ ]:
def avgFilter(img):
  mask = np.ones([3, 3], dtype=int)/9
  avg_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      for p in range(-1, 2):
        for q in range(-1, 2):
          avg_image[i, j] = avg_image[i, j] + img[i+p, j+q] * mask[p, q]
  return avg_image

In [ ]:
def medFilter(img):
  med_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      med_image[i, j] = sorted(temp)[4]
  return med_image

In [ ]:
def minFilter(img):
  min_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      min_image[i, j] = min(temp)
  return min_image

In [ ]:
def maxFilter(img):
  max_image = np.zeros(img.shape)
  for i in range(1, img.shape[0]-1):
    for j in range(1, img.shape[1]-1):
      temp = []
      for p in range(-1, 2):
        for q in range(-1, 2):
          temp = temp + [img[i+p, j+q]]
      max_image[i, j] = max(temp)
  return max_image

In [ ]:
def otsuThresh(img):
  # find normalized_histogram, and its cumulative distribution function
  hist = cv2.calcHist([img], [0], None, [256], [0,256])
  norm = hist.ravel() / hist.max()
  Q = norm.cumsum()

  bins = np.arange(256)
  minval = np.inf
  thresh = -1

  for i in range(1,256):
      prob1, prob2 = np.hsplit(norm, [i]) # probabilities

      cumsum1, cumsum2 = Q[i], Q[255] - Q[i]    # cumulative sum of classes
      if cumsum1 == 0 or cumsum2 == 0:
        continue
        
      weight1, weight2 = np.hsplit(bins, [i]) # weights

      # finding means
      mean1, mean2 = np.sum(prob1 * weight1) / cumsum1, np.sum(prob2 * weight2) / cumsum2

      # minimize the withinclass variance function
      res = np.sum((weight1 - mean1) ** 2 * prob1) + np.sum((weight2 - mean2) ** 2 * prob2)
      if res < minval:
          minval = res
          thresh = i
  img[img < thresh] = 0
  img[img != 0] = 255
  print(thresh)
  return img

In [ ]:
class Node:
  def __init__(self, frequency, char = None, left=None, right=None):
    self.left = left
    self.right = right
    self.frequency = frequency
    self.char = char
    self.direction = ""

  def printNode(self, val=""):
    newVal = val + str(self.direction)
    if self.left:
      self.left.printNode(newVal)
    if self.right:
      self.right.printNode(newVal)
    if not self.left and not self.right:
      print(f"{self.char}: {newVal}")


def HuffmanCoding(code):
  frequency = dict()

  for char in code:
    if char not in frequency.keys():
      frequency[char] =  0
    frequency[char] += 1

  queue = [Node(freq, char) for char, freq in sorted(frequency.items(), key=lambda item: item[1])]

  while len(queue) > 1:
    left = queue.pop(0)
    right = queue.pop(0)

    left.direction = "0"
    right.direction = "1"
    queue.append(Node(left.frequency+right.frequency, left.char+right.char, left, right))

    queue = sorted(queue, key= lambda x: x.frequency)
  root = queue[0]
  encodings = {x:y for x, y in queue[0].printNode()}
  print(code)
  encoded = ""
  for char in code:
    encoded += encodings[char]
  return encoded